In [1]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

# Parameter provided manually.
alert_id = "SA-AG-01"

# Parameters from the alert yaml.
label = "Voltage-gated sodium channel"
region = "2RL:63,000,000-65,000,000"
ir_candidate_genes = [
    dict(
        identifier="AGAP004707",
        names=["Vgsc", "Para"],
        notes="Blah blah blah.",
        references=[
            dict(
                citation="Clarkson et al. (2021)",
                title="FOo bar",
                url="https://doi.org/10.1111/mec.15845",
            ),
            dict(
                citation="Martinez-Torres et al. (1998)",
                title="Spam eggs",
                url="https://pubmed.ncbi.nlm.nih.gov/9535162/",
            ),
        ],
    ),
]

# Parameters from the workflow config.yaml.
cohorts_analysis = "20240924"
analysis_version = "2025.02.11"
dask_scheduler = "single-threaded"

In [2]:
# Parameters
analysis_version = "2025.02.13"
min_cohort_size = 15
max_cohort_size = 70
sample_sets = ["3.0"]
sample_query = (
    "unrestricted_use and taxon in ['gambiae', 'coluzzii', 'arabiensis', 'bissau']"
)
contigs = ["2RL", "3RL", "X"]
cohorts_analysis = "20240924"
h12_calibration_contig = "3L"
h12_signal_detection_min_delta_aic = 1000
h12_signal_detection_min_stat_max = 0.1
h12_signal_detection_gflanks = [6]
dask_scheduler = "single-threaded"
alerts = ["01", "02", "03", "04", "05", "06", "07", "08", "09"]
label = "Cytochrome P450 9K1"
region = "X:14,220,000-16,220,000"
ir_candidate_genes = [
    {
        "identifier": "AGAP000818",
        "names": ["Cyp9k1"],
        "notes": "P450 gene associated with pyrethroid resistance through increased expression and copy number variation.\n",
        "references": [
            {
                "citation": "Vontas et al. (2018)",
                "title": "Rapid selection of a pyrethroid metabolic enzyme CYP9K1 by operational malaria control activities\n",
                "url": "https://doi.org/10.1073/pnas.1719663115",
            }
        ],
    }
]
alert_id = "SA-AG-05"


In [3]:
from pyprojroot import here

root = here()
%run -i {root}/workflow/common/scripts/setup.py
%run -i {root}/workflow/site/scripts/page-setup.py

region_contig, region_span = region.split(":")
region_start, region_stop = region_span.replace(",", "").split("-")

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# Alert `SA-AG-05` (Cytochrome P450 9K1)

In [4]:
html = f"This alert reports selection signals on contig {region_contig} within the region {region_span} bp."
HTML(html)

HTML(value='This alert reports selection signals on contig X within the region 14,220,000-16,220,000 bp.')

## Selection signals

Selection signals overlapping this genome region are shown in the figure below.

In [5]:
df_signals = load_signals(
    contig=region_contig,
    start=region_start,
    stop=region_stop,
)

plot_signals(
    df=df_signals,
    contig=region_contig,
    x_min=df_signals["span2_pstart"].min() - 50_000,
    x_max=df_signals["span2_pstop"].max() + 50_000,
)

## Cohorts affected
Overlapping signals of selection are found in the following cohorts. 

In [6]:
cohort_links = [
    '<a href="../cohort/' + row["cohort_id"] + '.html">' + row["cohort_label"] + "</a>"
    for i, row in df_signals.iterrows()
]
html_message = "<li>" + "</li><li>".join(cohort_links) + "</li>"
HTML(html_message)

HTML(value='<li><a href="../cohort/GH-EP_New-Juaben-South-Municipal_gamb_2012_Q4.html">Ghana / New Juaben Sout…

## Insecticide resistance genes

The following genes are found within this genome region and may be driving recent positive selection based on evidence for an association with resistance to insecticides.

In [7]:
for gene in ir_candidate_genes:
    gene_id = gene["identifier"]
    html = f'<p><strong><a href="https://vectorbase.org/vectorbase/app/record/gene/{gene_id}">{gene_id}</a></strong>'
    gene_names = gene["names"]
    if gene_names:
        html += " (" + ", ".join(gene_names) + ")"
    html += " — "
    notes = gene["notes"]
    html += f"{notes} See also:"
    html += "<ul>"
    for reference in gene["references"]:
        citation = reference["citation"]
        url = reference["url"]
        title = reference["title"]
        html += f'<li><a href="{url}">{citation}</a> {title}</li>'
    html += "</ul></p>"
    display(HTML(html))

HTML(value='<p><strong><a href="https://vectorbase.org/vectorbase/app/record/gene/AGAP000818">AGAP000818</a></…